In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

# Other packages used in this vignette
packageVersion("Voyager")

# Introduction

Consider two variables that are correlated, say with Pearson correlation of 0.8. The observations are spatially referenced. The locations of the observations can be permuted without affecting Pearson correlation. The purpose of bivariate spatial statistics is to indicate both correlation in value (as in Pearson correlation), and spatial autocorrelation and co-patterning. 
One of the bivariate methods implemented in `Voyager` is the cross variogram, which is shown in the [variogram vignette](https://pachterlab.github.io/voyager/articles/variogram.html). This vignette demonstrates other bivariate spatial statistics, which use a spatial neighborhood graph, on the mouse skeletal muscle Visium dataset.

Here we load the packages used:

In [ ]:
library(Voyager)
library(SFEData)
library(SpatialFeatureExperiment)
library(scater)
library(scran)
library(ggplot2)
library(pheatmap)
library(scico)
theme_set(theme_bw())

A list of all bivariate global methods can be seen here:

In [ ]:
listSFEMethods(variate = "bi", scope = "global")

When calling `calculate*variate()` or `run*variate()`, the `type` (2nd) argument takes either an `SFEMethod` object or a string that matches an entry in the `name` column in the data frame returned by `listSFEMethods()`.

QC was performed in [another vignette](https://pachterlab.github.io/voyager/articles/vig1_visium_basic.html), so this vignette will not plot QC metrics.

In [ ]:
(sfe <- McKellarMuscleData("full"))

The image can be added to the SFE object and plotted behind the geometries, and needs to be flipped to align to the spots because the origin is at the top left for the image but bottom left for geometries.

In [ ]:
if (!file.exists("tissue_lowres_5a.jpeg")) {
    download.file("https://raw.githubusercontent.com/pachterlab/voyager/main/vignettes/tissue_lowres_5a.jpeg",
                  destfile = "tissue_lowres_5a.jpeg")
}

In [ ]:
sfe <- addImg(sfe, file = "tissue_lowres_5a.jpeg", sample_id = "Vis5A", 
              image_id = "lowres", 
              scale_fct = 1024/22208)
sfe <- mirrorImg(sfe, sample_id = "Vis5A", image_id = "lowres")

In [ ]:
sfe_tissue <- sfe[,colData(sfe)$in_tissue]
sfe_tissue <- sfe_tissue[rowSums(counts(sfe_tissue)) > 0,]

In [ ]:
sfe_tissue <- logNormCounts(sfe_tissue)

In [ ]:
colGraph(sfe_tissue, "visium") <- findVisiumGraph(sfe_tissue)

# Lee's L

Lee's L [@Lee2001-tm] was developed from relating Moran's I to Pearson correlation, and is defined as

$$
L_{X,Y} = \frac{n}{\sum_{i=1}^n \sum_{j=1}^n w_{ij}} \frac{\sum_{i=1}^n \left[ \sum_{j=1}^n w_{ij}  (x_j - \bar{x}) \right] \left[ \sum_{j=1}^n w_{ij} (y_j - \bar{y}) \right]}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2}\sqrt{\sum_{i=1}^n (y_i - \bar{y})^2} },
$$

where $n$ is the number of spots or locations, $i$ and $j$ are different locations, or spots in the Visium context, $x$ and $y$ are variables with values at each location, and $w_{ij}$ is a spatial weight, which can be inversely proportional to distance between spots or an indicator of whether two spots are neighbors, subject to various definitions of neighborhood. 

Here we compute Lee's L for top highly variagle genes (HVGs) in this dataset:

In [ ]:
hvgs <- getTopHVGs(sfe_tissue, fdr.threshold = 0.01)

Because bivariate global results can have very different formats (matrix for Lee's L and lists for many other methods), the results are not stored in the SFE object.

In [ ]:
res <- calculateBivariate(sfe_tissue, type = "lee", feature1 = hvgs)

This gives a spatially informed correlation matrix among the genes, which can be plotted as a heatmap:

In [ ]:
pal_rng <- getDivergeRange(res)
pal <- scico(256, begin = pal_rng[1], end = pal_rng[2], palette = "vik")

In [ ]:
pheatmap(res, color = pal, show_rownames = FALSE, 
         show_colnames = FALSE, cellwidth = 1, cellheight = 1)

Some coexpression blocks can be seen. Note that unlike in Pearson correlation, the diagonal is not 1, because

$$
L_{X,X} = \frac{\sum_i (\tilde x_i - \bar x)^2}{\sum_i (x_i - \bar x)^2} = \mathrm{SSS}_X,
$$

which is approximated the ratio between the variance of spatially lagged $x$ and variance of $x$. Because the spatial lag introduces smoothing, the spatial lag reduced variance, making the diagonal less than 1. This is the spatial smoothing scalar (SSS), and Moran's I is approximately Pearson correlation between $X$ and spatially lagged $X$ ($\tilde X$) multiplied by SSS:

$$
I \approx \mathrm{SSS}_X \cdot \rho_{X, \tilde X}
$$

Similarly for Lee's L, as shown in [@Lee2001-tm],

$$
L_{X, Y} = \sqrt{\mathrm{SSS}_X}\sqrt{\mathrm{SSS}_Y} \cdot \rho_{\tilde X, \tilde Y}
$$

With more spatial clustering, the variance is less reduced by the spatial lag, leading to a larger SSS. Hence when both $X$ and $Y$ are spatially distributed like salt and pepper while strongly correlated, Lee's L will be low because the lack of spatial autocorrelation leads to a small SSS. 

Weighted correlation network analysis (WGCNA) [@Langfelder2008-fs] is a time honored method to find gene co-expression modules, and it can take any correlation matrix. Then it would be interesting to apply WGCNA to the Lee's L matrix to identify spatially informed gene co-expression modules.

# Local Lee

Local Lee's L [@Lee2001-tm] is defined as

$$
L_i = \frac{n\left[ \sum_{j=1}^n w_{ij}  (x_j - \bar{x}) \right] \left[ \sum_{j=1}^n w_{ij} (y_j - \bar{y}) \right]}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2}\sqrt{\sum_{i=1}^n (y_i - \bar{y})^2} }
$$

Compare this to the global L in the previous section. Local L does not sum over the locations $i$. This is the contribution of each location to global L and can show spatial heterogeneity in the relationship between two variables. 

All bivariate local methods in `Voyager` is listed here:

In [ ]:
listSFEMethods("bi", "local")

Here we compute local L for two myofiber marker genes and one gene highly expressed in the injury site:

In [ ]:
sfe_tissue <- runBivariate(sfe_tissue, "locallee", swap_rownames = "symbol",
                           feature1 = c("Myh2", "Myh1", "Ftl1"))

Bivariate local results are stored in the `localResults` field and the feature names are the pairwise combinations of features supplied. When only `feature1` is specified, then the bivariate method is applied to all pairwise combinations of `feature1`. 

In [ ]:
localResultFeatures(sfe_tissue, "locallee")

For Lee's L, both $L_{X,Y}$ and $L_{Y,X}$ are computed although they are the same. However, not all bivariate methods are symmetric (see next section). In the next release (Bioconductor 3.18), we may introduce another argument to indicate whether the method is symmetric and if so only compute $L_{X,Y}$ and not $L_{Y,X}$.

First plot the three genes individually:

In [ ]:
plotSpatialFeature(sfe_tissue, c("Myh2", "Myh1", "Ftl1"), 
                   swap_rownames = "symbol", image_id = "lowres", maxcell = 5e4)

Then plot the local L's:

In [ ]:
plotLocalResult(sfe_tissue, "locallee", c("Myh1__Myh2", "Myh2__Ftl1", "Myh1__Ftl1"),
                colGeometryName = "spotPoly",
                image_id = "lowres", maxcell = 5e4,
                divergent = TRUE, diverge_center = 0)

Here we see regions where Myh1 and Myh2 are more co-expressed, and where the myosins and Ftl1 are negatively correlated. 

$L_{X,X}$ is also computed, so we can plot the local SSS for the three genes:

In [ ]:
plotLocalResult(sfe_tissue, "locallee", c("Myh1__Myh1", "Myh2__Myh2", "Ftl1__Ftl1"),
                colGeometryName = "spotPoly",
                image_id = "lowres", maxcell = 5e4)

See how the local SSS compares to local Moran's I:

In [ ]:
sfe_tissue <- runUnivariate(sfe_tissue, "localmoran", c("Myh2", "Myh1", "Ftl1"),
                            swap_rownames = "symbol")

In [ ]:
plotLocalResult(sfe_tissue, "localmoran", c("Myh1", "Myh2", "Ftl1"),
                colGeometryName = "spotPoly", swap_rownames = "symbol",
                image_id = "lowres", maxcell = 5e4,
                divergent = TRUE, diverge_center = 0)

The patterns are qualitatively the same, but while local Moran's I is negative in heterogeneous regions, the SSS can't be negative.

# Bivariate local Moran

The `spdep` package implements a bivariate version of local Moran, which basically is 

$$
I_{X_i,Y_i} = (n-1)\frac{(x_i - \bar{x})\sum_{j=1}^n w_{ij} (y_j - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}.
$$

Note that this is not symmetric, i.e. $I_{X_i,Y_i} \neq I_{Y_i,X_i}$.

In [ ]:
sfe_tissue <- runBivariate(sfe_tissue, "localmoran_bv", c("Myh1", "Myh2", "Ftl1"),
                           swap_rownames = "symbol", nsim = 1000)

In [ ]:
localResultFeatures(sfe_tissue, "localmoran_bv")

Permutation testing is performed so we get a pseudo p-value

In [ ]:
localResultAttrs(sfe_tissue, "localmoran_bv", "Myh1__Myh2")

First plot the bivariate local Moran's I values

In [ ]:
plotLocalResult(sfe_tissue, "localmoran_bv", c("Myh1__Myh2", "Myh2__Ftl1", "Myh1__Ftl1",
                                               "Myh2__Myh1", "Ftl1__Myh2", "Ftl1__Myh1"),
                colGeometryName = "spotPoly", attribute = "Ibvi",
                image_id = "lowres", maxcell = 5e4,
                divergent = TRUE, diverge_center = 0)

The first row plots XY while the second row plots YX; note that while they are similar, they are not the same. What does bivariate local Moran mean? It's kind of like contribution of each location to the correlation between $x$ and spatially lagged $y$, so $x$ is not smoothed. In contrast, Lee's L is a scaled Pearson correlation between spatially lagged $x$ and spatially lagged $y$. Because permutation testing is performed, we can plot the pseudo-p-value, after correcting for multiple testing based on the spatial neighborhood graph:

In [ ]:
plotLocalResult(sfe_tissue, "localmoran_bv", c("Myh1__Myh2", "Myh2__Ftl1", "Myh1__Ftl1",
                                               "Myh2__Myh1", "Ftl1__Myh2", "Ftl1__Myh1"),
                colGeometryName = "spotPoly", attribute = "-log10p_adj Sim",
                image_id = "lowres", maxcell = 5e4,
                divergent = TRUE, diverge_center = -log10(0.05))

Note that the p-values are asymetric, because according to the source code of `localmoran_bv()`, $y$ is permuted, but not $x$. It's also related to Wartenberg's spatial PCA [@Wartenberg1985-fk], where Moran's I is expressed in matrix form:

$$
\mathbf{I} = \frac{\mathbf{Z}^T\mathbf{WZ}}{\mathbf 1^T \mathbf{W1}},
$$

where $\mathbf Z$ is the data matrix with scaled and centered variables in columns, $\mathbf W$ is the spatial weights matrix, and $\mathbf 1$ is a vector of all 1's, so the denominator is in effect $\sum_{i=1}^n \sum_{j=1}^n w_{ij}$. The diagonal entries are Moran's I's for the variables, and the off diagonal entries are the global versions of what we computed here that sum the bivariate local Moran's I's and divide by the sum of all spatial weights. Because $\mathbf W$ doesn't have to be symmetric, this matrix may not be symmetric. Wartenberg diagonalized this matrix in place of the covariance matrix for spatial PCA. When using scaled and centered data and row normalized spatial weights matrix, MULTISPATI PCA is equivalent to Wartenberg's approach [@Dray2008-en]. Lee considered this asymmetry an inadequacy of Wartenberg's approach as a bivariate association measure [@Lee2001-tm]. While I'm not sure how bivariate local Moran's I helps with data analysis, it is an interesting piece of history. 

# Session info

In [ ]:
sessionInfo()

# References